<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Context Compression

"Compression" is a bit of a strange term to use here, but the basic gist is that using another LLM call we can ask the model to extract only the relevant information from a vector doc lookup call. 

Let's revisit our last example:

In [1]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

### OpenAI Connection for Embeddings

In [2]:
import os
embedding_function = OpenAIEmbeddings()

In [3]:
# docs

### Connect to Embed Documents via ChromaDB

In [4]:
db_connection = Chroma(persist_directory='./mk_ultra',embedding_function=embedding_function)

### Contextual Compression

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [6]:
llm = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [7]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db_connection.as_retriever())

In [8]:
docs = db_connection.similarity_search('When was this declassified?')

In [9]:
docs[0]

Document(page_content='The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee\'s efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.\nThe most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance

In [10]:
compressed_docs = compression_retriever.get_relevant_documents("When was this declassified?")

/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [11]:
compressed_docs[0].page_content

'The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS).'